### LSTM model tuner
### Long Short Term Memory

* Define functions
* try to find the best hyperparameters
* Train the model

Importing libraries

In [157]:
import pandas as pd
from matplotlib import pyplot

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Input
from keras.callbacks import TensorBoard
from keras.layers import Dropout
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

from math import sqrt
from numpy import concatenate

In [158]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5, restore_best_weights=True)

Define functions

In [159]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [160]:
tf.compat.v1.keras.layers.CuDNNLSTM

def lstm_model_builder(hp):

    hp_learning_rate = hp.Choice('learning_rate', values=['1e-2', '1e-3', '1e-4'])

    model = Sequential()
    
    model.add(LSTM(hp.Int('input_unit',
                        min_value=4,
                        max_value=512,
                        step=8), 
                        return_sequences=False, 
                        input_shape=(train_x.shape[1],train_x.shape[2])
                        ))

    # for i in range(hp.Int('n_layers', 1, 4)):
    #     model.add(LSTM(hp.Int(f'lstm_{i}_units',min_value=32,max_value=512,step=4),return_sequences=True))

    # model.add(LSTM(hp.Int('layer_2_neurons',min_value=32,max_value=512,step=4)))
    # model.add(Dropout(hp.Float('Dropout_rate',min_value=0,max_value=0.5,step=0.1)))
    model.add(Dense(train_y.shape[1], activation=hp.Choice('dense_activation',values=['relu', 'sigmoid'],default='relu')))
    model.compile(loss='mean_squared_error', optimizer='adam',metrics = ['mse', 'mae'])
        
    return model

Load data

In [161]:
df = pd.read_parquet("../data/processed/new-avc.parquert")
df.head()

,date,co_ppb,o3_ppb,pm25_ugm3,so2_ugm3,temperatura_c,umidade_relativa_percentual,precipitacao_mmdia,focos_queimada,respiratory,avc
0,2013-01-01,135.40,11.125,8.175,0.875,26.450,81.75,2.0,0.0,6.0,1.0
1,2013-01-02,155.05,10.925,11.675,1.000,26.400,82.00,2.0,0.0,8.0,0.0
2,2013-01-03,137.40,12.225,8.800,0.825,27.400,81.25,0.0,0.0,12.0,1.0
3,2013-01-04,128.30,13.750,11.650,0.925,27.750,76.25,1.0,0.0,9.0,0.0
4,2013-01-05,128.25,10.950,7.800,1.075,26.625,84.75,9.0,0.0,10.0,1.0


Normalize features

In [162]:
df = df.set_index("date")

In [163]:
columns_to_drop = ["co_ppb", "so2_ugm3", "precipitacao_mmdia", "focos_queimada", "avc"]

In [164]:
# df = df.drop(columns=["avc"])
df = df.drop(columns=columns_to_drop)

In [165]:
values = df.values

In [166]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

Apply series_to_supervised

In [167]:
n_lags = 3

In [168]:
df_sl = series_to_supervised(scaled, n_in=n_lags, n_out=1)

Drop columns I don't want to predict

In [169]:
df_sl.drop(df_sl.columns[[15, 16, 17, 18]], axis=1, inplace=True)

Define and fit model

Split into train and test sets

In [170]:
values = df_sl.values
train_set, test_set = train_test_split(
    df_sl, test_size=0.2, shuffle=False
)

In [171]:
train_X, train_Y = train_set.drop("var5(t)", axis=1), train_set.drop(train_set.columns[:15], axis=1)
test_X, test_Y = test_set.drop("var5(t)", axis=1), test_set.drop(test_set.columns[:15], axis=1)

Reshape input to be 3D [samples, timesteps, features]

In [172]:
train_x, test_x = train_X.values, test_X.values
train_y, test_y = train_Y.values, test_Y.values

In [173]:
train_x = train_x.reshape((train_x.shape[0], 1, train_x.shape[1]))
test_x = test_x.reshape((test_x.shape[0], 1, test_x.shape[1]))

In [174]:
tuner= RandomSearch(
        lstm_model_builder,
        objective='mse',
        max_trials=384,
        executions_per_trial=2,
        project_name = "2_lstm"
        )

INFO:tensorflow:Reloading Oracle from existing project ./2_lstm/oracle.json
INFO:tensorflow:Reloading Tuner from ./2_lstm/tuner0.json


In [175]:
tuner.search(
        x=train_x,
        y=train_y,
        epochs=50,
        batch_size=128,
        validation_data=(test_x,test_y),
        callbacks=[TensorBoard(log_dir="logs/LSTM/2_tuner"), callback]
)

INFO:tensorflow:Oracle triggered exit


In [176]:
best_model = tuner.get_best_models(num_models=1)[0]

In [177]:
yhat = best_model.predict(test_x)
test_x = test_x.reshape((test_x.shape[0], test_x.shape[2]))
yhat = yhat.reshape((yhat.shape[0], yhat.shape[1]))

14/14 [==============================] - 0s 7ms/step


In [178]:
# invert scaling for forecast
inv_yhat = concatenate((test_x[:, -4:], yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,4]
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_x[:, -4:], test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,4]
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

Test RMSE: 3.239
